In [1]:
import os
from ensure import ensure_annotations

In [2]:
os.chdir("../")

In [3]:
!pwd

/Users/ngkuissi/Dev/Image_Search_Engine


In [11]:
from dataclasses import dataclass
from pathlib import Path
from typing import List

@dataclass(frozen=True)
class PredictionConfig:
    model_path: Path
    image_path_list_dir: Path
    n_neighbors: int
    return_distance: bool
    include_top: bool
    pooling: str
    input_shape: List
    target_size: List

In [12]:
from imageSearchEngine.constants import *
from imageSearchEngine.utils.file_helpers import read_yaml, create_directories

In [13]:
class ConfigurationManager:
    @ensure_annotations
    def __init__(
        self,
        config_filepath:Path = CONFIG_FILE_PATH,
        params_filepath:Path = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    @ensure_annotations
    def get_prediction_config(self) -> PredictionConfig:
        config = self.config.prediction
        params = self.params.prediction
        create_directories([config.root_dir])

        prediction_config = PredictionConfig(
            model_path= config.model_path,
            image_path_list_dir = config.image_path_list_dir,
            n_neighbors=params.n_neighbors,
            return_distance= params.return_distance,
            include_top= params.include_top,
            pooling = params.pooling,
            input_shape= params.input_shape,
            target_size= params.target_size
        )

        return prediction_config

In [19]:
import os
from tqdm import tqdm
import pickle
from PIL import Image
from sklearn.neighbors import KNeighborsClassifier
from tensorflow.keras.utils import load_img, img_to_array
from tensorflow .keras.applications import resnet50
from tensorflow.image import resize
import numpy as np

from imageSearchEngine.logging.logger import log
from imageSearchEngine.exception import CustomException

In [22]:
class Prediction:

    def __init__(self, config: PredictionConfig):
        self.config = config


    def extract(self, image):
        model = resnet50.ResNet50(
            include_top=self.config.include_top,
            input_shape=self.config.input_shape,
            pooling=self.config.pooling
        )
        image = img_to_array(image)
        image = np.expand_dims(image, axis=0)
        image = resize(image, self.config.target_size)
        preprocess_image = resnet50.preprocess_input(image)
        feature = model.predict(preprocess_image, verbose=0).flatten()
        return feature
        

    
    def predict(self, image) -> List:
        im = Image.open(image)
        feature = self.extract(im)
        image_path_list = pickle.load(open(self.config.image_path_list_dir, 'rb'))
        model: KNeighborsClassifier = pickle.load(open(self.config.model_path, 'rb'))
        index_list = model.kneighbors(feature.reshape(1, -1),
                                        n_neighbors=self.config.n_neighbors, return_distance=self.config.return_distance
                                        )[0]
        prediction_path = [image_path_list[i] for i in  index_list]
        return prediction_path



In [23]:
try:
    configuration = ConfigurationManager()
    model_evaluation_config =  configuration.get_prediction_config()
    model_evaluation = Prediction(config=model_evaluation_config)
    print(model_evaluation.predict("artifacts/data_ingestion/tiny-imagenet-200/val/n01629819/images/val_3592.JPEG"))
except Exception as e:
    raise CustomException(e)

[2023-10-10 18:53:55,345: INFO: file_helpers: yaml file: config/config.yaml loaded successfully]
[2023-10-10 18:53:55,348: INFO: file_helpers: yaml file: params.yaml loaded successfully]
[2023-10-10 18:53:55,349: INFO: file_helpers: created directory at: artifacts]
[2023-10-10 18:53:55,349: INFO: file_helpers: created directory at: artifacts/predictions]
['artifacts/data_ingestion/tiny-imagenet-200/train/n01629819/images/n01629819_440.JPEG', 'artifacts/data_ingestion/tiny-imagenet-200/train/n01629819/images/n01629819_10.JPEG', 'artifacts/data_ingestion/tiny-imagenet-200/train/n01629819/images/n01629819_273.JPEG', 'artifacts/data_ingestion/tiny-imagenet-200/train/n01629819/images/n01629819_362.JPEG', 'artifacts/data_ingestion/tiny-imagenet-200/train/n01629819/images/n01629819_45.JPEG', 'artifacts/data_ingestion/tiny-imagenet-200/train/n01629819/images/n01629819_186.JPEG', 'artifacts/data_ingestion/tiny-imagenet-200/train/n03444034/images/n03444034_191.JPEG', 'artifacts/data_ingestion/ti